In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import numpy as np
import pandas as pd

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()
        
        if num_layers == 1:
            dropout = 0

        # Embedding Layer
        self.embedding = nn.Embedding(input_size, embedding_size)

        # Recurrent Layer
        if cell_type == "LSTM":
            self.recurrent_layer = nn.LSTM(embedding_size, hidden_size, num_layers, dropout, batch_first=True)
        elif cell_type == "GRU":
            self.recurrent_layer = nn.GRU(embedding_size, hidden_size, num_layers, dropout, batch_first=True)
        else: # Default (RNN)
            self.recurrent_layer = nn.RNN(embedding_size, hidden_size, num_layers, dropout, batch_first=True)
    
    def forward(self, input, input_lengths):
        # input: batch_size x seq_len
        # input_lengths: length of each sequence

        embeddings = self.embedding(input)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, num_layers=1, cell_type="RNN", dropout=0.0):
        super().__init__()
        pass

    def forward(self):
        pass

In [ ]:
class Seq2Seq(nn.Module):
    def __init__():
        pass

    def forward():
        pass

### Prepare Data

In [ ]:
# Load train, dev and test datasets
train_df = pd.read_csv('dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv', sep='\t', header=None)
dev_df = pd.read_csv('dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv', sep='\t', header=None)
test_df = pd.read_csv('dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv', sep='\t', header=None)


# Rename the columns appropriately
train_df.columns = ['native_script', 'latin', 'frequency'] 
dev_df.columns = ['native_script', 'latin', 'frequency'] 
test_df.columns = ['native_script', 'latin', 'frequency'] 

# Dataset Sizes
print(f"Train Dataset Size : {train_df.shape[0]}\nDev Dataset Size   : {dev_df.shape[0]}\nTest Dataset Size  : {test_df.shape[0]}")

Train Dataset Size : 44204
Dev Dataset Size   : 4358
Test Dataset Size  : 4502


In [18]:
train_df.head()

,native_script,romanized,frequency
0,अं,an,3
1,अंकगणित,ankganit,3
2,अंकल,uncle,4
3,अंकुर,ankur,4
4,अंकुरण,ankuran,3
